In [ ]:
import os
import sys
import numpy as np
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
#import tensorflow

In [2]:
from skimage import io
from skimage.transform import resize
from numpy import array
from sklearn.model_selection import train_test_split

In [7]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
import pickle
import csv
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf

In [5]:
batch_size = 4
n_classes = 5
input_shape = 512,512,3

In [4]:
trainLabels = pd.read_csv("/Volumes/surya/aptos2019-blindness-detection/train.csv")
trainLabels.head()

,id_code,diagnosis
0,000c1434d8d7,2
1,001639a390f0,4
2,0024cdab0c1e,1
3,002c21358ce6,0
4,005b95c28852,0


In [ ]:
img_rows, img_cols = 512, 512
from skimage import color

path = '/Volumes/surya/aptos2019-blindness-detection/train_images/'
immatrix = []
imlabel = []
dirs = [l for l in os.listdir(path) if l != '.DS_Store']
total = 0
for item in dirs:
    base = os.path.basename(path + item)
    #print(base)
    fileName = os.path.splitext(base)[0]
    #print(fileName)
    total +=1
    #print(dirs)
    try:
      ls = trainLabels.loc[trainLabels.id_code==fileName, 'diagnosis']
      lb = trainLabels.values[ls]
      ld = imlabel.append(ls)
      print(imlabel)
      print(fileName," ",total)
      import cv2
      import glob
      import imageio
      from skimage.color import rgb2gray
      #imgs = glob.glob('/content/train_images/*.png')
      # for i in imgs:
      #   img = cv2.imread(i)
      #   resize_img = cv2.resize(img, (512, 512))
      #   cv2.imwrite(i, resize_img)
      img = cv2.imread(path +item)
      resize_img = cv2.resize(img, (512,512))
      img_gray = cv2.cvtColor(np.asarray(img), cv2.COLOR_BGR2GRAY)
      img_gray.flatten()
      cv2.imwrite("gray"+'/Volumes/surya/aptos2019-blindness-detection/train_images/*.png',img_gray)
      pixels = img_gray.astype(np.float32)
      immatrix.append(pixels)
      #plt.imshow(img)
    except (IOError,ValueError) as e:
      print('could not read the',fileName ,':',e,'hence skipping it.')

In [ ]:
immatrix = np.asarray(immatrix)
imlabel = np.asarray(imlabel)

batch_size = 16
n_classes = 5
input_shape = 512, 512, 1

from sklearn.utils import shuffle

data,Label = shuffle(immatrix,imlabel, random_state=4)
train_data = [data,Label]
type(train_data)

(X, y) = (train_data[0],train_data[1])

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2)

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols,1)
x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols,1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols,1)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_val /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

from keras.utils import np_utils

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, n_classes)
y_val = np_utils.to_categorical(y_val, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)

#data augmentation
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
datagen = ImageDataGenerator(   
        rotation_range=10, 
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        shear_range=0.15,  
        zoom_range=0.1,  
        fill_mode='nearest', 
        horizontal_flip=True)
datagen.fit(x_train)

train_gen = datagen.flow(x_train, y_train, batch_size=batch_size)

#DenseNet Model

import keras
from keras_applications.densenet import DenseNet121
from keras.layers import *
from keras import regularizers
model = DenseNet121(include_top = False,
                    input_shape= input_shape,
                  weights=None,
                  backend=keras.backend,
                  layers=keras.layers,
                  models=keras.models,
                  utils=keras.utils)
x = model.output
x = GlobalAveragePooling2D()(x)
x =Dropout(0.3)(x)
x = Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(0.01))(x)
out = Dense(n_classes, activation='softmax')(x)

from keras import Model
model=Model(inputs=model.input,outputs=out)
model.summary()

from keras.optimizers import Adam, SGD
adam=Adam(lr=0.00003, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.load_weights("weights.den_1st.hdf5")
print("loaded")

# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

model_history_dn_aug = model.fit_generator(train_gen, 
                              steps_per_epoch=len(x_train)//batch_size, 
                              epochs=10, verbose=1,
                              validation_data = (x_val,y_val))

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss {:.4f}, accuracy {:.2f}%'.format(scores[0], scores[1] * 100))

model.save_weights("weights.den_1st.hdf5")
print("loaded")

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
pred_Y = model.predict(x_test)
pred_Y = np.argmax(pred_Y, axis=1)
test_Y = np.argmax(y_test, axis=1)
print(classification_report(test_Y, pred_Y))

cnf_matrix = (confusion_matrix(test_Y, pred_Y))
print("Confusion matrix \n",cnf_matrix)